In [1]:
from skmultilearn.dataset import load_dataset
from sklearn.model_selection import train_test_split
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import re
import random
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import os
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import matplotlib.pyplot as plt
from skmultilearn.model_selection import iterative_train_test_split
from skmultilearn.dataset import load_from_arff
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss, accuracy_score, f1_score, precision_score, recall_score
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import sklearn.metrics as metrics
from torch.utils.data import DataLoader, Subset
import itertools
import pickle
from sklearn.preprocessing import MinMaxScaler
from skmultilearn.dataset import load_dataset
from skmultilearn.model_selection import IterativeStratification
from sklearn.neighbors import NearestNeighbors
import pdb
import scipy
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from enum import Enum
import math
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import roc_auc_score
from torch.optim.lr_scheduler import LambdaLR
from util import *
from layers import  *
from model import *
from function import *
from scipy.stats import pearsonr
from collections import deque
from scipy.stats import gaussian_kde
from sklearn.neighbors import KernelDensity
from scipy.integrate import dblquad
from joblib import Parallel, delayed
from sklearn.feature_selection import mutual_info_classif
from scipy.stats import entropy

In [2]:
#####seed####
def seed_all(seed): 
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
seed=42
seed_all(seed)
device = torch.device("cuda:0" ) if torch.cuda.is_available() else torch.device("cpu")

In [3]:
class CFG:
    def __init__(self,name,X,y):
        self.name=name      
        self.X_train=X
        self.y_train=y
        self.configs={} 
    def getconfig(self):
        self.configs['label_matrix']=np.array(self.y_train)
        self.configs['num_classes']=self.y_train.shape[1] 
        self.configs['num_ins']=self.X_train.shape[0] 
        self.configs['seed']=42 
        self.configs['batch_size']=128
        self.configs['epoch']=100 
        self.configs['lr']=1e-4
        self.configs['device']=torch.device("cuda:0" ) if torch.cuda.is_available() else torch.device("cpu")
        self.configs['weight']=limb(np.array(self.X_train),np.array(self.y_train))
        self.configs['extra_sample']=int(self.X_train.shape[0]*0.1)
        self.configs['min_ins_idx']=minority_instance(np.array(self.y_train))
        self.configs['minority_label_indices'],_=Labeltype(np.array(self.y_train))
        self.configs['weight_list']=calweight(np.array(self.X_train),np.array(self.y_train))
        self.configs['card'],_=CardAndDens(np.array(self.X_train),np.array(self.y_train))
        #DELA
        if self.name=='DELA':
            self.configs['in_features']=self.X_train.shape[1] 
            self.configs['latent_dim']=math.ceil(self.X_train.shape[1]/2)    
            self.configs['lr_ratio']=0.8
            self.configs['drop_ratio']=0.2
            self.configs['tau']=2/3
            self.configs['beta']=1e-4
            self.configs['out_index']=-1
        if self.name=='CLIF':
            self.configs['class_emb_size']=self.y_train.shape[1]  
            self.configs['input_x_size']=self.X_train.shape[1] 
            self.configs['num_layers']=2 
            self.configs['in_layers']=3 
            self.configs['hidden_list']=[math.ceil(self.y_train.shape[1]/2)]  
            self.configs['out_index']=0        
        if self.name=='PACA':
            self.configs['drop_ratio']=0.1
            self.configs['latent_dim']=math.ceil(self.X_train.shape[1]/2)
            self.configs['in_features']=self.X_train.shape[1] #输入x的维度
            self.configs['rand_seed']=self.configs['seed']
            self.configs['eps']=1e-8    
            self.configs['lr_scheduler']='fix'
            self.configs['binary_data']=False
            self.configs['weight_decay']=1e-5
            self.configs['alpha']=2
            self.configs['gamma']=10
            self.configs['scheduler_warmup_epoch']=5
            self.configs['scheduler_decay_epoch']=10
            self.configs['scheduler_decay_rate']=1e-5 
            self.configs['out_index']=-2
        return self.configs

In [4]:
seed_all(seed)
device = torch.device('cuda')
def FeatureSelect(X,p):
    if p==1:
        return X.toarray(),feature_names
    else:
        featurecount=int(X.shape[1]*p)
        Selectfeatureindex=[x[0] for x in (sorted(enumerate(X.sum(axis=0).tolist()[0]),key=lambda x: x[1],reverse=True))][:featurecount]
        Allfeatureindex=[i for i in range(X.shape[1])]
        featureindex=[i for i in Allfeatureindex if i not in Selectfeatureindex]
        new_x=np.delete(X.toarray(),featureindex,axis=1)
        new_featurename=[feature_names[i] for i in Selectfeatureindex] 
        return new_x,new_featurename
def LabelSelect(y):
    b=[]
    new_labelname=[i for i in label_names]
    for i in range(y.shape[1]):
        if y[:,i].sum()<=20:
            b.append(i)
            new_labelname.remove(label_names[i])
    new_y=np.delete(y.toarray(),b,axis=1)
    return new_y,new_labelname
def macro_averaging_auc(Y, P, O):
    n = (Y.shape[0] + O.shape[0]) // 2
    l = (Y.shape[1] + O.shape[1]) // 2

    p = np.zeros(l)
    q = np.sum(Y, 0)

    zero_column_count = np.sum(q == 0)
#     print(f"all zero for label: {zero_column_count}")
    r, c = np.nonzero(Y)
    for i, j in zip(r, c):
        p[j] += np.sum((Y[ : , j] < 0.5) * (O[ : , j] <= O[i, j]))

    i = (q > 0) * (q < n)

    return np.sum(p[i] / (q[i] * (n - q[i]))) / l
def hamming_loss(Y, P, O):
    n = (Y.shape[0] + P.shape[0]) // 2
    l = (Y.shape[1] + P.shape[1]) // 2

    s1 = np.sum(Y, 1)
    s2 = np.sum(P, 1)
    ss = np.sum(Y * P, 1)

    return np.sum(s1 + s2 - 2 * ss) / (n * l)
def one_error(Y, P, O):
    n = (Y.shape[0] + O.shape[0]) // 2

    i = np.argmax(O, 1)

    return np.sum(1 - Y[range(n), i]) / n
def ranking_loss(Y, P, O):
    n = (Y.shape[0] + O.shape[0]) // 2
    l = (Y.shape[1] + O.shape[1]) // 2

    p = np.zeros(n)
    q = np.sum(Y, 1)

    r, c = np.nonzero(Y)
    for i, j in zip(r, c): 
        p[i] += np.sum((Y[i, : ] < 0.5) * (O[i, : ] >= O[i, j]))

    i = (q > 0) * (q < l)

    return np.sum(p[i] / (q[i] * (l - q[i]))) / n
def micro_f1(Y, P, O):
    return f1_score(Y, P, average='micro')
def macro_f1(Y, P, O):
    return f1_score(Y, P, average='macro')
def eval_metrics(mod, metrics, datasets, idx,batch_size,device):
    res_dict = {}
    mod.eval()
    y_true_list = []
    y_scores_list = []
    test_dataloader = DataLoader(datasets, batch_size=batch_size, shuffle=True, num_workers=0)
    for x, y in test_dataloader:
        _, y_pred=mod.predict(x)
#         pdb.set_trace()
        y_true_list.append(y.cpu().numpy())
        y_scores_list.append(y_pred.cpu().numpy())
    y_true = np.vstack(y_true_list)
    y_prob = np.vstack(y_scores_list)
    y_pred = np.round(y_prob).astype(int)
    res_dict1 = {metric.__name__: metric(y_true, y_pred,y_prob) for metric in metrics}
#         # Calculate metric.
#         res_dict1 = {metric.__name__: metric(y_true, y_pred) for metric in metrics[:2]}
#         res_dict2 = {metric.__name__: metric(y_true, y_prob) for metric in metrics[2:5]}
#         res_dict1.update(res_dict2)
#     res_dict[f'dataset_{ix}']=res_dict1
#         res_dict[f'dataset_{ix}'] = {metric.__name__: metric(y_true, y_pred) for metric in metrics[:8]}
#         res_dict[f'dataset_{ix}'] = {metric.__name__: metric(y_true, y_prob) for metric in metrics[8:9]}
    return res_dict1

In [32]:
def update_H(H, y_pred, ids, max_history_length=5):
    y_pred_numpy = y_pred.detach().cpu().numpy()
    for i, idx in enumerate(ids):
        if idx not in H:
            H[idx] = deque(maxlen=max_history_length) 
        H[idx].append(y_pred_numpy[i])  
    return H
def update_E(H,E,ids,label_dim):
    
    for idx in ids:
        current_predictions_history = np.array(H[idx])
        last_row_index = len(current_predictions_history) - 1
        for j in range(label_dim): 
            diffs = np.abs(np.diff(current_predictions_history[:, j]))
            mean_diffs = np.sum(diffs)/len(diffs)
            current_entropy = -1 / np.log(2) * (current_predictions_history[last_row_index][j]  * np.log(current_predictions_history[last_row_index][j]) + (1 - current_predictions_history[last_row_index][j] ) * np.log(1 -current_predictions_history[last_row_index][j] ))
            E[idx,j]=1/2*mean_diffs+1/2* current_entropy
    return E

def update_U(E, U, epoch):
    E[E > 1] = 1
    if epoch>=5:
        bins = np.array([0, 0.2, 0.4, 0.6, 0.8, 1.0])
        discrete_values = np.array([0.1, 0.3, 0.5, 0.7, 0.9])
        indices = np.digitize(E, bins) - 1
        indices = np.clip(indices, 0, len(discrete_values) - 1)
        E = discrete_values[indices]
#         I=np.abs(calculate_pearson_matrix(E))
        
    else:
        I = np.ones((E.shape[1],E.shape[1]))
#     if epoch==30:
#         np.save('yahoo-Business1I(30).npy', I)
    I = np.ones((E.shape[1],E.shape[1]))
    U = np.dot(E, I)
    w = np.sum(U, axis=1)
    return w


def calculate_probabilities(U, epoch, e_end, e0, se0):
    # Delta is set to 1/N
    N=len(U)
    Delta = 1 / N 
    # Exponentially decay the selection pressure as training progresses
    se = se0 * (np.exp(np.log(1 / se0) / (e_end - e0)))**(epoch - e0)
    U_min = np.min(U)
    U_max = np.max(U)
    U_normalized = (U - U_min) / (U_max - U_min)
    quantized_indices = np.ceil((1 - U_normalized) / Delta).astype(int)
#     pdb.set_trace()
    # Calculate the exponent for the sampling probability
    exponent = -np.log(se) / N * quantized_indices 
    # Calculate the unnormalized probabilities
    unnormalized_probabilities = np.exp(exponent)
    return unnormalized_probabilities


def calculate_mutual_information_matrix(data):
    n, q = data.shape
    mi_matrix = np.zeros((q, q))
    for i in range(q):
        for j in range(q):
            if i == j:
                mi_matrix[i, j] = 1  # 对角线置为1
            else:
                mi_matrix[i, j] = mutual_info_classif(data[:, i].reshape(-1, 1), data[:, j])[0]
                mi_matrix[i, j] = max(mi, 0)
    return mi_matrix

def calculate_pearson_matrix(data):
    pearson_matrix = np.corrcoef(data, rowvar=False)
    return pearson_matrix

def entropy_based_mutual_information_matrix(data):
    n, q = data.shape
    mi_matrix = np.zeros((q, q))
    
    for i in range(q):
        for j in range(i, q):
            if i == j:
                mi_matrix[i, j] = 1
            else:
                mi = entropy_based_mutual_information(data[:, i], data[:, j])
                mi_matrix[i, j] = mi
                mi_matrix[j, i] = mi
    
    # Normalize each row by its maximum value
    row_max = mi_matrix.max(axis=1, keepdims=True)
    mi_matrix = mi_matrix / row_max
    
    return mi_matrix

def entropy_based_mutual_information(x, y):
    # Calculate mutual information between x and y using entropy-based methods
    # Ensure x and y are flattened to 1D arrays
    x = np.ravel(x)
    y = np.ravel(y)
    
    # Calculate individual entropies
    h_x = entropy(np.histogram(x, bins='auto')[0])
    h_y = entropy(np.histogram(y, bins='auto')[0])
    
    # Calculate joint entropy
    joint_hist, _, _ = np.histogram2d(x, y, bins='auto')
    joint_entropy = entropy(joint_hist.ravel())
    
    # Mutual information
    mi = h_x + h_y - joint_entropy
    return mi

class CustomDataLoader:
    def __init__(self, dataset, net, configs,H,E,U,sample_probabilities,warm_epoch):
        self.shuffle = False
        self.dataset = dataset
        self.net = net
        self.batch_size = configs['batch_size']
        self.sample_probabilities =sample_probabilities
        self.dataset_indices=list(range(len(self.dataset)))
        self.warm_epoch=warm_epoch   
        
        self.H=H
        self.E=E
        self.U=U    
                              
    def __iter__(self):
        if self.current_epoch < self.warm_epoch:
            batch_counter = 0
            for start_idx in range(0, len(self.dataset_indices), self.batch_size):
                end_idx = min(start_idx + self.batch_size, len(self.dataset_indices))
                batch_indices = self.dataset_indices[start_idx:end_idx]
                batch_counter += 1
                yield self.get_data_from_indices(batch_indices)
        else:
            all_indices = self.dataset_indices  
            all_probabilities = self.sample_probabilities   

            total_sum = all_probabilities.sum()
            all_probabilities /= total_sum

            total_samples = len(self.dataset)
            num_batches = np.ceil(total_samples / self.batch_size).astype(int)

            for _ in range(num_batches):
                num_required = self.batch_size 
                if num_required > 0:
                    chosen_indices = np.random.choice(all_indices, size=num_required, replace=False, p=all_probabilities)
                else:
                    chosen_indices = []
                batch_indices = list(chosen_indices)
                yield self.get_data_from_indices(batch_indices)
                
    def get_data_from_indices(self, indices):
        x, y = zip(*[self.dataset[i] for i in indices])
        return indices, torch.stack(x), torch.stack(y)
    def set_epoch(self, epoch):
        self.current_epoch = epoch

In [ ]:
def training(configs,warm_epoch):
    net=CLIFModel(configs).to(device)
    num_epochs =configs['epoch']
    batch_size = configs['batch_size']
    card=int(configs['card'])
#     card=configs['num_classes']
    lr = 1e-4
    
    ins_dim=configs['num_ins']
    label_dim=configs['num_classes']
    
    weight_decay = 1e-4  
    betas = (0.9, 0.999) 

    optimizer = torch.optim.Adam(net.parameters(), lr=lr,  
                           betas=betas, weight_decay=weight_decay)
    writer = SummaryWriter(comment=f'{dataname}')

    sample_probabilities=[]
    
    sample_indices=list(range(ins_dim))
    H = {idx: deque(maxlen=5) for idx in sample_indices}
    E= np.zeros((ins_dim, label_dim))
    U = np.zeros(ins_dim)
    
    
    custom_dataloader = CustomDataLoader(train_dataset, net=net, configs=configs,H=H,E=E,U=U,sample_probabilities=sample_probabilities,warm_epoch=warm_epoch)
    validation_dataset, test_dataset_new = train_test_split(test_dataset, test_size=0.5, random_state=42)   
    test_dataloader = DataLoader(test_dataset_new, batch_size=batch_size, shuffle=False, num_workers=0)
    validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    best_auc = 0
    best_model_state = None
    epoch_losses_train = []

    warmup_epochs =warm_epoch
    total_steps = num_epochs * int(configs['num_ins']/batch_size)
    warmup_steps = warmup_epochs * int(configs['num_ins']/batch_size)
    global_step = 0
    def update_learning_rate(optimizer, global_step, warmup_steps=warmup_steps, base_lr=lr):
        if global_step < warmup_steps:
            lr = base_lr * (global_step / warmup_steps)
        else:
            lr = base_lr
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr 
            
    pcorrelation_matrix=np.zeros((label_dim, label_dim))
    for epoch in range(num_epochs+1): 
        net.train()
        batch_counter = 0   
        loss_tracker = 0.0
        loss_tracker2=0.0
        all_individual_losses = {}
        custom_dataloader.set_epoch(epoch) 
        for idx, x, y in custom_dataloader:
            optimizer.zero_grad()
            outputs = net(x)
            loss_dict = net.loss_function_train(outputs, y)
            loss = loss_dict['Loss']
            loss.backward()
            optimizer.step()
            global_step += 1
            update_learning_rate(optimizer, global_step) 
            custom_dataloader.H=update_H(custom_dataloader.H, torch.sigmoid(outputs[0]), idx)
            custom_dataloader.E=update_E(custom_dataloader.H, custom_dataloader.E,idx,label_dim)
            batch_counter+=1 
        custom_dataloader.U=update_U(custom_dataloader.E, custom_dataloader.U, epoch)
        custom_dataloader.sample_probabilities=calculate_probabilities(custom_dataloader.U, epoch, 10, 100, 8)    
#         custom_dataloader.E=update_E(custom_dataloader.H, custom_dataloader.E,idx,label_dim)
#         pcorrelation_matrix=calculate_pearson_correlation(custom_dataloader.E)
        writer.add_scalar('train/loss', loss_tracker, epoch)
        epoch_losses_train.append(loss_tracker /batch_counter)

        net.eval()
        y_true_list = []
        y_scores_list = []
        loss_tracker = 0.0
        batch_counter = 0
        with torch.no_grad():
            for x, y in validation_dataloader:
                _, y_pred=net.predict(x)
                y_true_list.append(y.cpu().numpy())
                y_scores_list.append(y_pred.cpu().numpy())
                
                outputs = net(x)
                loss_dict = net.loss_function_train(outputs, y)
                loss = loss_dict['Loss']
                loss_tracker += loss.item()
                batch_counter += 1
        y_true = np.vstack(y_true_list)
        y_scores = np.vstack(y_scores_list)

        auc = macro_averaging_auc(y_true,y_scores, y_scores)
#         writer.add_scalar('val/auc', auc, epoch)

        if auc > best_auc:
            best_auc = auc
            best_epoch=epoch
            best_model_state = net.state_dict().copy()
    net.load_state_dict(best_model_state)
    mets = eval_metrics(net, [macro_f1, micro_f1, macro_averaging_auc, ranking_loss, hamming_loss, one_error], test_dataset_new, configs['out_index'],configs['batch_size'],torch.device('cuda'))
    return mets

In [ ]:
path_to_arff_files = ["emotions","scene","yeast", "Corel5k","rcv1subset1","rcv1subset2","rcv1subset3","yahoo-Business1","yahoo-Arts1","bibtex",'tmc2007','enron','cal500','LLOG-F']
label_counts = [6, 6,14,374,101,101,101,28,25,159,22,53,174,75]
select_feature=[1,1,1,1,0.02,0.02,0.02,0.05,0.05,1,0.01,1,1,1]

for idx, dataname in enumerate(path_to_arff_files):
    path_to_arff_file = f"/home/tt/{dataname}.arff"
    X, y, feature_names, label_names = load_from_arff(
        path_to_arff_file,
        label_count=label_counts[idx],
        label_location="end",
        load_sparse=False,
        return_attribute_definitions=True
        )
    X,feature_names=FeatureSelect(X,select_feature[idx])  
    y,label_names=LabelSelect(y)
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    warm_epoch=10
    print(dataname)
    k_fold = IterativeStratification(n_splits=5,order=1,random_state=42)
    dicts=[]
    for idx,(train,test) in enumerate(k_fold.split(X,y)):
        train_dataset = TensorDataset(torch.tensor(X[train], device=device, dtype=torch.float),torch.tensor(y[train], device=device,dtype=torch.float))
        test_dataset = TensorDataset(torch.tensor(X[test], device=device, dtype=torch.float), torch.tensor(y[test], device=device, dtype=torch.float))    
        configs=CFG('CLIF',X[train],y[train]).getconfig()
        dict_1=training(configs,warm_epoch)
        dicts.append(dict_1)
    averages_and_stds = {}
    for key in dicts[0].keys():
        values = [d[key] for d in dicts]
        averages_and_stds[key] = {
            'average': round(np.mean(values),4),
            'std': round(np.std(values),4)
        }
    print(averages_and_stds)